In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from sparkmeasure import StageMetrics
from pyspark.sql.functions import lower,col

In [2]:
spark = SparkSession \
    .builder \
    .appName('Db Project') \
    .config("spark.jars", "/spark-measure_2.12-0.17.jar") \
    .getOrCreate()

In [3]:
spark

In [7]:
stagemetrics = StageMetrics(spark)

In [5]:
movies = (spark.read
           .format('csv')
           .option('header', 'true')
           .option('delimiter', ',')
           .option('inferSchema', 'true')
           .load('movie.csv'))

rating = (spark.read
           .format('csv')
           .option('header', 'true')
           .option('delimiter', ',')
           .option('inferSchema', 'true')
           .load('rating.csv'))

tag = (spark.read
           .format('csv')
           .option('header', 'true')
           .option('delimiter', ',')
           .option('inferSchema', 'true')
           .load('tag.csv'))

In [6]:
tag = tag.withColumn('tag', lower(col('tag')))

In [7]:
movies.createOrReplaceTempView('movies')
rating.createOrReplaceTempView('ratings')
tag.createOrReplaceTempView('tags')

## Query 1

In [14]:
stagemetrics.begin()
spark.sql("SELECT COUNT(userId) AS users FROM ratings WHERE movieId = \
                    (SELECT movieId FROM movies WHERE title='Jumanji (1995)')").show()
stagemetrics.end()

+-----+
|users|
+-----+
|22243|
+-----+



In [ ]:
stagemetrics.print_report()

## Query 2

In [15]:
stagemetrics.begin()
spark.sql("SELECT DISTINCT(title) FROM movies INNER JOIN tags ON tags.movieId=movies.movieID WHERE tag='boring' ORDER BY 1 ASC LIMIT 5").show(truncate=0)
stagemetrics.end()

+------------------------------------+
|title                               |
+------------------------------------+
|(500) Days of Summer (2009)         |
|101 Reykjavik (101 Reykjavík) (2000)|
|12 Years a Slave (2013)             |
|1408 (2007)                         |
|1492: Conquest of Paradise (1992)   |
+------------------------------------+



In [ ]:
stagemetrics.print_report()

## Query 3 

In [16]:
stagemetrics.begin()
spark.sql("SELECT DISTINCT(ratings.userId) FROM ratings INNER JOIN tags ON tags.movieId = ratings.movieId \
                      AND tags.userId = ratings.userId WHERE tag='bollywood' AND rating>3 ORDER BY ratings.userId").show(5)
stagemetrics.end()

+------+
|userId|
+------+
| 10573|
| 19837|
| 23333|
| 25004|
| 31338|
+------+
only showing top 5 rows



In [ ]:
stagemetrics.print_report()

## Query 4

In [17]:
stagemetrics.begin()
spark.sql("SELECT title,AVG(rating) FROM movies INNER JOIN ratings ON movies.movieId=ratings.movieId WHERE timestamp LIKE '2005%' GROUP BY 1 ORDER BY 2 DESC, movies.title LIMIT 10").show(10,truncate=0)
stagemetrics.end()

+----------------------------------------------------------------------------+-----------+
|title                                                                       |avg(rating)|
+----------------------------------------------------------------------------+-----------+
|Before the Fall (NaPolA - Elite für den Führer) (2004)                      |5.0        |
|Dancemaker (1998)                                                           |5.0        |
|Fear Strikes Out (1957)                                                     |5.0        |
|Gate of Heavenly Peace, The (1995)                                          |5.0        |
|Life Is Rosy (a.k.a. Life Is Beautiful) (Vie est belle, La) (1987)          |5.0        |
|Married to It (1991)                                                        |5.0        |
|My Life and Times With Antonin Artaud (En compagnie d'Antonin Artaud) (1993)|5.0        |
|Not Love, Just Frenzy (Más que amor, frenesí) (1996)                        |5.0        |

In [ ]:
stagemetrics.print_report()

## Query 5

In [22]:
stagemetrics.begin()
spark.sql("SELECT movies.title as title, collect_set(tags.tag) AS tags FROM movies INNER JOIN tags ON tags.movieId = movies.movieId WHERE tags.timestamp LIKE '%2015%' GROUP BY title ORDER BY title").show(5)
stagemetrics.end()

+--------------------+--------------------+
|               title|                tags|
+--------------------+--------------------+
|""Great Performan...|              [bd-r]|
|  'burbs, The (1989)|[1980's, dark com...|
|(500) Days of Sum...|[zooey deschanel,...|
|...tick... tick.....|              [bd-r]|
|            1 (2014)|           [sukumar]|
+--------------------+--------------------+
only showing top 5 rows



In [ ]:
stagemetrics.print_report()

## Query 6

In [ ]:
stagemetrics.begin()
spark.sql("SELECT title,COUNT(rating) FROM movies JOIN ratings ON movies.movieId=ratings.movieId GROUP BY 1 ORDER BY 2 DESC").show(5,truncate=0)
stagemetrics.end()

In [ ]:
stagemetrics.print_report()

## Query 7

In [ ]:
stagemetrics.begin()
spark.sql("SELECT userId, COUNT(rating) FROM ratings WHERE SUBSTRING(timestamp,1,4) = '1995' GROUP BY userId, SUBSTRING(timestamp,1,4)").show()
stagemetrics.end()

In [ ]:
stagemetrics.print_report()

## Query 8

In [23]:
stagemetrics.begin()
spark.sql("SELECT genres, first(title) as title, MAX(count) as total_ratings FROM (SELECT movies.genres, movies.title, COUNT(ratings.rating) AS count FROM movies INNER JOIN ratings ON ratings.movieId = movies.movieId GROUP BY movies.genres, movies.title ORDER BY genres, count DESC) GROUP BY genres ORDER BY genres").show(5,truncate=0)
stagemetrics.end()

+-----------------------------------+------------------------------------------------------------------------------+-------------+
|genres                             |title                                                                         |total_ratings|
+-----------------------------------+------------------------------------------------------------------------------+-------------+
|(no genres listed)                 |Doctor Who: The Time of the Doctor (2013)                                     |36           |
|Action                             |Under Siege 2: Dark Territory (1995)                                          |8335         |
|Action|Adventure                   |Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)|43295        |
|Action|Adventure|Animation         |How to Train Your Dragon 2 (2014)                                             |678          |
|Action|Adventure|Animation|Children|Brave (2012)                                  

In [ ]:
stagemetrics.print_report()

## Query 9

In [24]:
stagemetrics.begin()
spark.sql("SELECT SUM(total) FROM (SELECT COUNT(userId) as total FROM ratings GROUP BY movieId, SUBSTRING(timestamp,6,9) HAVING total > 1 ORDER BY total DESC)").show()
stagemetrics.end()

+----------+
|sum(total)|
+----------+
|  12003573|
+----------+



In [ ]:
stagemetrics.print_report()

# Query 10

In [25]:
stagemetrics.begin()
spark.sql("SELECT movies.genres, COUNT(ratings.movieId) FROM ratings INNER JOIN movies ON movies.movieId = ratings.movieId INNER JOIN tags ON tags.movieId = ratings.movieId WHERE ratings.rating > 3.5 AND tags.tag LIKE '%funny%' GROUP BY movies.genres ORDER BY genres").show(5, truncate=0)
stagemetrics.end()

+--------------------------------------------------+--------------+
|genres                                            |count(movieId)|
+--------------------------------------------------+--------------+
|Action|Adventure                                  |2140          |
|Action|Adventure|Animation|Children|Comedy        |89936         |
|Action|Adventure|Animation|Children|Comedy|Fantasy|1472          |
|Action|Adventure|Animation|Children|Comedy|IMAX   |410           |
|Action|Adventure|Animation|Children|Comedy|Sci-Fi |366           |
+--------------------------------------------------+--------------+
only showing top 5 rows



In [ ]:
stagemetrics.print_report()